# COGS 108 - Final Project (change this to your project's title)

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that student names will be included (but PIDs will be scraped from any groups who include their PIDs).

* [  ] YES - make available
* [  ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Ant Man
- Hulk
- Iron Man
- Thor
- Wasp

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

<a id='background'></a>

## Background & Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

The projects we analyzed were publicly available on github and to our knowledge the projects are only public if the students explicitly agree for the projects to be public. If a student of a project dissents to its analysis due to perhaps not forseeing the possibility that their project would be looked at, then they probably should not have agreed to make it public in the first place. As for the students' privacy, the names of the students were available on every project we analyzed. However, we only scraped the projects of its word count, topics, etc. and never touched or used the names for anything in our research. Our project not once mentions these names or calls attention to certain individuals or projects. So while there is room for privacy concerns regarding usage of names, we do not make the identities any less private than they already are. 

The biggest concern with bias in our analysis rests on the API we used for deciding topics after parsing the introductory paragraphs. It could classify up to 560 suprisingly specific topics, but there are obviously much more topics in existence. So we lose a lot of nuance with what the topic of the project is during analysis. We lose even more nuance when you consider that if for example a project topic is car racing, it could be read by the API to be about automobiles if the paragraph's contents is 60% cars and 40% sports, completely cutting out how the topic is 40% about sports and therefore treated equally as a topic that is about trucks. We originally accounted for this by switching from an initial API to the much better and more specific API we currently use, but beyond that there is not much more we can realistically do beyond checking the hundreds of projects by hand as API's are not at a human level of nuance yet. The best we can do is acknowledge that any analysis is not set in stone as some topic trends may be inflated by unique but somewhat similar topics.

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*